<h1>Data Preprocessing</h1>
<li>Steps that need to be done after the data arrives in the program
<li>Data may need to be cleaned
<ul>
<li>Remove "bad" data
<li>handle "missing" data 
</ul>
<li>Make sure all data is in the correct format(s)


<h2>NYC 311 Data</h2>
<li>Service requests made to the 311 number</li>
<li>https://data.cityofnewyork.us/dataset/311-Service-Requests-From-2011/fpz8-jqf4
<li>We'll work with a small extract of the data
<ul>
<li>8 columns (out of about 58 in the full dataset)
<li>two months of data (out of 7 years of data
</ul>
<li>Even this is quite a bit!

In [ ]:
datafile = "./Class 6 - nyc_311_data_subset_small.csv"

In [ ]:
import pandas as pd
import numpy as np

<h4><b>read_csv</b>: A pandas function that reads a comma separated file</h4>
read_csv will try to format the data so that it is the correct type and will report any typing problems<br>
It will also look for a header row. 
<br>http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html
<p>
read_csv returns a pandas DataFrame

In [ ]:
df = pd.read_csv(datafile)
df

<h4>Let's examine our data</h4>

In [ ]:
df.info()

<h4>Looks like Unique Key really is a unique key and can serve as an index</h4>

In [ ]:
df = pd.read_csv(datafile,index_col='Unique Key')

In [ ]:
df.iloc[:10]

<h4>Columns 4 has mixed types</h4>

<h4>Column 4 is incident zip</h4>
Let's examine it<br>
The unique() function returns unique values in a column

In [ ]:
df['Incident Zip'].unique()

<h4>Some issues</h4>
<li>Sometimes zip is a float, other times it is an str
<li>Zipcodes that are represented as floats and start with 0 are missing the first digit (8817)
<li>Some zipcodes have the 4 digit extension added. Comparison becomes tough
<li>What is zip 0?
<li>What about the missing (nan) values? The ? (question mark)? "UNKNOWN"?

<h2>Clean the data</h2>
<li>Decide what to do with "bad" data ("JFK", "UNKNOWN", etc.). Convert to Nan or delete the record.
<li>Make sure all data in a column is in the correct format (convert floats to strings, get rid of the 4 digit extension)
<li>Decide what to do with missing values (NaNs)

<h3>for "Incident Zip"</h3>
<li>we'll drop rows with NaN or bad data
<li>get rid of the 4 digit extension
<li>remove zips less than 10000 and greater than 12000
<li>Let's write a function that fixes zips
<li>Convert everything to str because zips are not really numbers

In [ ]:
def fix_zip(input_zip):
    try:
        input_zip = int(float(input_zip))
    except:
        try:
            input_zip = int(input_zip.split('-')[0])
        except:
            return np.NaN
    if input_zip < 10000 or input_zip > 12000:
        return np.NaN
    return str(input_zip)
        

<h4>And test it</h4>

In [ ]:
fix_zip('10817.00')

<h3>Apply the function to input zip</h3>
<li>The pandas function <span style="color:blue">apply</span> applies a function to each element in a df column


In [ ]:
df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)

In [ ]:
df['Incident Zip'].unique()

<h3>Drop all rows that have zip == Nan</h3>
<li>We don't have to, that's just a choice we're making</li>
<li>Alternatively, we could just give them a special code</li>
<li>Or vary the code depending on some other factor (Borough? department?)

In [ ]:
df = df[df['Incident Zip'].notnull()]

In [ ]:
df

<h3>Let's take a look at the columns again</h3>

In [ ]:
df.info()

<h3>Closed Date, Latitude and Longitude all have missing values</h3>
<h3>Let's get rid of them</h3>

In [ ]:
df = df[(df['Latitude'].notnull()) & (df['Longitude'].notnull())  & (df['Closed Date'].notnull())]

In [ ]:
df.info()

<h4>Let's take a look at Borough data</h4>

In [ ]:
df['Borough'].unique()

<h4>Unspecified</h4>
<li>Could be a problem if our analysis focuses on boroughs

In [ ]:
df[df['Borough']=='Unspecified'][['Agency','Incident Zip']]

<h3>Possible fixes</h3>
<li>Map zips to boroughs using external data
<li>Map zips to boroughs using data from the data frame
<li>Delve into the data to see whether unspecified makes sense

<h4>So we get to use our groupby function</h4>

In [ ]:
df[df['Borough']=='Unspecified'].groupby('Agency').count()

<li>Unspecified appears to have a systematic bias toward NYPD
<li>Though only a small proportion of NYPD complaints (see below)
<li>We have to decide whether to keep them or lose them!

In [ ]:
nypd_complaints_total = df[df['Agency']=='NYPD']['Borough'].count()
nypd_unspecified = df[(df['Borough']=='Unspecified') & (df['Agency']=="NYPD")]['Borough'].count()
percentage = nypd_unspecified/nypd_complaints_total*100
print("%1.2f%%"%percentage)

<h4>For now, we'll get rid of them. Unspecified will be hard to explain!</h4>

In [ ]:
df = df[df['Borough'] != 'Unspecified']

<h2>Dealing with time</h2>
<li>Dates and times are best converted to datetime
<li>That way they will be useful for analysis because we can compute timedelta objects

<h2>Digression: The datetime library</h2>

In [ ]:
df.info()

<h4>And we get to use lambda functions!</h4>

In [ ]:
import datetime
df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))

In [ ]:
df['Created Date'][0:20]

In [ ]:
df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))

In [ ]:
df

<h3>We can create a new column that tracks the time it takes to close a complaint</h3>

In [ ]:
df['processing_time'] =  df['Closed Date'] - df['Created Date']

In [ ]:
#And look at summary statistics
df['processing_time'].describe()

In [ ]:
df['Created Date'].min()

<h4>There is some odd stuff here</h4>
<li>Negative processing time?
<li>Since our data is for five months, a max of 148 days worth checking out

<h3>Let's examine the negative processing time data</h3>

In [ ]:
df[df['processing_time']<datetime.timedelta(0,0,0)]

In [ ]:
df[df['processing_time']<datetime.timedelta(0,0,0)].count()

<h4>Possibilities</h4>
<li>Data entry errors
<li>Created Date and Closed Date got switched


<h3>And the large processing times as well</h3>

In [ ]:
df[df['processing_time']>datetime.timedelta(148,0,0)]

In [ ]:
df[df['processing_time']>datetime.timedelta(148,0,0)].count()

<li>Looks like the upper end makes sense but the negative times don't
<li>Though we need to explore this more, we'll get rid of negative times for now

In [ ]:
df = df[df['processing_time']>=datetime.timedelta(0,0,0)]

<h2>Finally, let's write a function that incorporates all our changes</h2>

In [ ]:
def read_311_data(datafile):
    import pandas as pd
    import numpy as np
    #Add the fix_zip function
    def fix_zip(input_zip):
        try:
            input_zip = int(float(input_zip))
        except:
            try:
                input_zip = int(input_zip.split('-')[0])
            except:
                return np.NaN
        if input_zip < 10000 or input_zip > 19999:
            return np.NaN
        return str(input_zip)
    
    #Read the file
    df = pd.read_csv(datafile,index_col='Unique Key')
    
    #fix the zip
    df['Incident Zip'] = df['Incident Zip'].apply(fix_zip)
    
    #drop all rows that have any nans in them (note the easier syntax!)
    
    df = df.dropna(how='any')
    
    #get rid of unspecified boroughs
    df = df[df['Borough'] != 'Unspecified']
    
    #Convert times to datetime and create a processing time column
    
    import datetime
    df['Created Date'] = df['Created Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['Closed Date'] = df['Closed Date'].apply(lambda x:datetime.datetime.strptime(x,'%m/%d/%Y %I:%M:%S %p'))
    df['processing_time'] =  df['Closed Date'] - df['Created Date']
    
    #Finally, get rid of negative processing times and return the final data frame
    
    df = df[df['processing_time']>=datetime.timedelta(0,0,0)]
    
    return df
    

In [ ]:
df = read_311_data('Class 6 - nyc_311_data_subset_small.csv')
df.info()

<h1>Summary</h1>
<li>We read the data using read_csv
<li>Cleaned Incident Zip
<li>Converted Dates to datetime
<li>Created a new column containing processing times


<h3>The data is ready!</h3>